In [27]:
#Importing necessary libraries
import os
import numpy as np
import pandas as pd
import pyodbc


In [28]:
#Pandas Dataframe datatype to SQL datatype replacements mapping
dtype_replacements = {
    "object" : 'varchar(30)',
    "float64" : "float",
    "int64" : "int",
    "datetime64" : "timestamp" ,
    "timedelta64[ns]" : "varchar(30)"
}
dtype_replacements

{'object': 'varchar(30)',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar(30)'}

In [29]:
#Iterating every csv files in the local drive path and pre-processing data

path_name = r"D:\\Udemy Courses\Bhavcopy"
files_in_dir = os.listdir(path_name)
print(files_in_dir)


for filename in files_in_dir:
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(path_name,filename))
        
        #replacing the column headers to lowercase and inappropriate naming conventions
        df.columns = [x.lower().replace(" ",'').replace("1",'') for x in df.columns]
        print(df.columns)
        print(df.dtypes)
        
         #Handling incosistent data in column names in csv files.
        df = df.replace(' -','')
        
        #generating sql table name with appropriate datatype based on CSV file for the columns
        sql_table_cols = ", ".join("{} {}".format(name,dtype) for (name,dtype) in zip(df.columns,df.dtypes.replace(dtype_replacements)))
        df.to_csv(os.path.join(path_name,filename), index= False, encoding = 'utf-8')
        print(sql_table_cols)
        
#os.path.join()

['sec_bhavdata_full_02042024.csv', 'sec_bhavdata_full_03042024.csv', 'sec_bhavdata_full_04042024.csv', 'sec_bhavdata_full_05042024.csv', 'sec_bhavdata_full_08042024.csv', 'sec_bhavdata_full_09042024.csv', 'sec_bhavdata_full_10042024.csv', 'sec_bhavdata_full_11042024.csv', 'sec_bhavdata_full_12042024.csv']
Index(['symbol', 'series', 'date', 'prev_close', 'open_price', 'high_price',
       'low_price', 'last_price', 'close_price', 'avg_price', 'ttl_trd_qnty',
       'turnover_lacs', 'no_of_trades', 'deliv_qty', 'deliv_per'],
      dtype='object')
symbol            object
series            object
date              object
prev_close       float64
open_price       float64
high_price       float64
low_price        float64
last_price        object
close_price      float64
avg_price        float64
ttl_trd_qnty       int64
turnover_lacs    float64
no_of_trades       int64
deliv_qty        float64
deliv_per        float64
dtype: object
symbol varchar(30), series varchar(30), date varchar(30), pr

PermissionError: [Errno 13] Permission denied: 'D:\\\\Udemy Courses\\Bhavcopy\\sec_bhavdata_full_05042024.csv'

In [12]:
#Table structure to be created
create table nse_dailydata_bhav_copy
(
    symbol            varchar,
    series            varchar,
    date              varchar,
    prev_close        float,
    open_price        float,
    high_price        float,
    low_price         float,
    last_price        float,
    close_price       float,
    avg_price         float,
    ttl_trn_qnty      int,
    turnover_lacs     float,
    no_of_trades      int,
    deliv_qty         int,
    deliv_per         float    
);

SyntaxError: invalid syntax (3393649726.py, line 2)

In [22]:
#Listing available PYODBC drivers
pyodbc.drivers()

['SQL Server',
 'Amazon Redshift (x64)',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [30]:
#Establishing and Open DB connectivity
driver_name = "SQL Server Native Client RDA 11.0"
server_name = "DESKTOP-EQ55Q8H"
db_name = 'NSEBhavcopy'


cnxn = pyodbc.connect(f"""
    Driver={{SQL Server}};
    Server={server_name};
    Database={db_name};
    """)

cursor = cnxn.cursor()
print ('opened database successfully')


opened database successfully


In [22]:
#Dropping same tables if exists on DB

cursor.execute("drop table if exists nse_dailydata_bhav_copy;")

In [20]:
#Creating table 
cursor.execute ("create table nse_dailydata_bhav_copy \
                (symbol varchar(30), series varchar(30), date varchar(30), prev_close float, open_price float, \
                high_price float, low_price float, last_price varchar(30), close_price float, avg_price float, \
                ttl_trd_qnty int, turnover_lacs float, no_of_trades int, deliv_qty float, deliv_per float)")

In [36]:

try:
    # here we can use with statement to automatically close connection once the operation is complete
    with cursor:
        for filename in files_in_dir:
            if filename.endswith('.csv'):
                    sql_stmt = f"""
                                BULK INSERT nse_dailydata_bhav_copy FROM '{os.path.join(path_name,filename)}'
                                WITH (
                                FORMAT = 'CSV',
                                FIRSTROW = 2,
                                FIELDTERMINATOR  = ',',
                                ROWTERMINATOR = '0x0a'
                                )
                                """.strip()
                    cursor.execute(sql_stmt)
                    #print(os.path.join(path_name,filename), nse_dailydata_bhav_copy + ' inserted')

                 
        cursor.commit()
except Exception as e:
    print(e)
    cursor.rollback()
    print('Transaction rollback')